### Question: What types of data are in a given ArcGIS REST service?

##### I wanted a way to summarize a large ArcGIS REST service. This example uses the Texas Department of Transportation, which has over 500 Feature Services. 

*still a work in progress

In [2]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
from arcgis.gis import GIS
from arcgis.features import FeatureLayer, summarize_data

In [4]:
# basics for bs4
url = 'https://services.arcgis.com/KTcxiTD9dsQw4r7Z/arcgis/rest/services/' # TxDOT 
html = urlopen(url)
soup = BeautifulSoup(html, 'lxml')

In [5]:
# some bs4 stuff
# find the anchor tag, grab the url and title, then zip the list together 
all_links = soup.find_all('a')
all_href = [link.get('href') for link in all_links]
all_title = [link.contents[0] for link in all_links]
href_and_title_list = list(zip(all_href, all_title))

In [6]:
# remove non-feature services, prefix the URL, and read into a df
df = pd.DataFrame(href_and_title_list[4:], columns=['URL', 'service_title'])
df['URL'] = df['URL'].apply(lambda x: f"https://services.arcgis.com/{x}")

# need to remove bad urls, then get a list of urls
df_clean =  df[~df.URL.str.contains('%', na = False)]
url_list = df_clean['URL'].tolist()
len(url_list)

539

In [7]:
# get the service service ItemId for each feature service
# with a lot of services, this can take a while...

# start with converting each url into a FeatureLayer
image_lyr_list = [FeatureLayer(url) for url in url_list]

# get the itemID 
image_ids_list = [feature_layer.properties.serviceItemId for feature_layer in image_lyr_list]
len(image_ids_list)




539

In [130]:
# add the itemID back into the clean df
df_clean['itemID'] = image_ids_list
df_clean.head()

/home/andresen/miniconda3/envs/python3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,URL,service_title,itemID
0,https://services.arcgis.com//KTcxiTD9dsQw4r7Z/...,2008_SealCoat,793e9dd1b22c4d7a94ff512e2ac4e39d
1,https://services.arcgis.com//KTcxiTD9dsQw4r7Z/...,2009SealCoat,7079ddda209c45e5b9fcf4baabf2929f
2,https://services.arcgis.com//KTcxiTD9dsQw4r7Z/...,2010MS4_UA,d01314cf386b4833affae229883ac8a5
3,https://services.arcgis.com//KTcxiTD9dsQw4r7Z/...,2010SealCoat,cc998298c7ab4b5d9ecf5b3bb5b6e4d5
4,https://services.arcgis.com//KTcxiTD9dsQw4r7Z/...,2012SealCoat,28fc8f08cc9b4f9ca56616b9e0182b71


In [1]:
# next, work on a workflow to access the data

item_id = 'd01314cf386b4833affae229883ac8a5'
gis = GIS()
gis_item = gis.content.get(item_id)
gis_item

NameError: name 'GIS' is not defined